#Importing Colab and necessary packages

In [1]:
from google.colab import drive
drive.mount("/content/drive")

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip install torchtext -qq
!pip install transformers evaluate wandb datasets accelerate -U -qq
!pip install emoji cleantext -qq

In [ ]:
import random
import numpy as np
import joblib
from collections import Counter
from pathlib import Path
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
import base64
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import PreTrainedModel, PretrainedConfig
from transformers import TrainingArguments, Trainer
from functools import partial
import wandb
import matplotlib.pyplot as plt
from datasets import load_dataset, DatasetDict
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoConfig
from transformers import pipeline
import re
from cleantext import clean
import emoji

##Path and defining folders

In [ ]:
basepath = ("/content/drive/MyDrive/Applied_NLP/")
sys.path.append("/content/drive/MyDrive/Applied_NLP/Custom_Files/")
sys.path

In [ ]:
basefolder = Path(basepath)
datafolder = basefolder/"Class_7/Home_Work_5/Data/"
modelfolder = basefolder/"Class_8/Home_Work_6/Model_Checkpoint/distil-bert-without-emoji"
customfolder = basefolder/"Custom_Files/"

In [ ]:
from CustomPreprocessorSpacy import SpacyPreprocessor

In [ ]:
datafolder

In [ ]:
from datasets import load_dataset

#Importing Data

In [ ]:
df_train = pd.read_csv(datafolder/"emoticons_train.csv")

In [ ]:
df_test = pd.read_csv(datafolder/"emoticons_test.csv")

##Labels in the data

In [ ]:
labels = [label for label in df_train.columns if label not in ["Tweet","ID"]]
id2labels = {idx:label for idx, label in enumerate(labels)}
labels2id = {v:k for k,v in id2labels.items()}
labels2id

#EDA of the classes

In [ ]:
def dist_class(df):
    df = df.iloc[:,2:]
    ones_count = df.sum()
    zeroes_count = (df == 0).sum()
    plt.figure(figsize=(15, 9))
    ones_count.plot(kind='bar', color='blue', position=0.5, width=0.4, label='Ones')
    zeroes_count.plot(kind='bar', color='orange', position=-0.5, width=0.4, label='Zeros')
    plt.title('Counts of Ones and Zeros for Each Class')
    plt.xlabel('Classes')
    plt.ylabel('Count')
    plt.xticks(rotation=0)
    plt.legend()
    plt.show()

In [ ]:
dist_class(df_train)

## EDA for each class

In [ ]:
def frequency(x):
  freq = df_train[x].value_counts(ascending=True)
  fig = freq.plot.barh()
  fig.set_ylabel('class frequency')
  fig.set_xlabel(x)
  plt.show()

In [ ]:
for x in labels:
  frequency(x)

#Dropping,Preprocessing and Splitting

In [ ]:
df_train.drop("ID",axis=1,inplace=True)
df_test.drop("ID", axis=1, inplace=True)

In [ ]:
df_train["Tweet"] = [emoji.replace_emoji(text, replace='') for text in df_train["Tweet"]]
#df_train["Tweet"] = [re.sub(r'@\w+','', text)for text in df_train["Tweet"]]
df_test["Tweet"] = [emoji.replace_emoji(text, replace='') for text in df_test["Tweet"]]
#df_test["Tweet"] = [re.sub(r'@\w+', '', text)for text in df_test["Tweet"]]

##Spacy Preprocessing

In [ ]:
sp = SpacyPreprocessor(model='en_core_web_sm', batch_size=64, lemmatize=True, lower=True,
                                    remove_stop=True, remove_punct=True, remove_email=True,
                                    remove_url=True, remove_num=False, stemming=False,
                                    add_user_mention_prefix=False, remove_hashtag_prefix=True, basic_clean_only=False)

In [ ]:
X_train_total = df_train["Tweet"].tolist()
X_test = df_test["Tweet"].tolist()

In [ ]:
y_test = df_train

In [ ]:
y_train_total = df_train.drop("Tweet",axis=1).to_numpy().astype(float)
y_test = df_test.drop("Tweet",axis=1).to_numpy()

In [ ]:
y_train_total

In [ ]:
y_test = np.where(y_test == "NONE",0,y_test).astype(float)

##Train-Valid Split

In [ ]:
X_train,X_valid,y_train,y_valid = train_test_split(X_train_total,y_train_total,test_size=0.3, random_state=42, shuffle=True)

##Spacy preprocessor

In [ ]:
X_train_preprocessed = sp.fit_transform(X_train)
X_valid_preprocessed = sp.transform(X_valid)
X_test_preprocessed = sp.transform(X_test)

##Data Dictionaries

In [ ]:
trainset = Dataset.from_dict({
    'texts': X_train_preprocessed,
    'labels': y_train
})

validset = Dataset.from_dict({
    'texts': X_valid_preprocessed,
    'labels': y_valid
})

testset = Dataset.from_dict({
    'texts': X_test_preprocessed,
    'labels': y_test
})

In [ ]:
trainset[0:5]

In [ ]:
testset[0:5]

In [ ]:
trainset.features

#Distil-bert as Checkpoint

In [ ]:
checkpoint_distil_bert = "distilbert-base-uncased"
tokenizer_distil_bert = AutoTokenizer.from_pretrained(checkpoint_distil_bert)

##Tokenize for distil-bert



In [ ]:
def tokenize_fn_bert(batch):
  return tokenizer_distil_bert(text = batch["texts"], truncation=True, padding="max_length",return_tensors = "pt", max_length = 128)

In [ ]:
tokenized_dataset_train_bert = trainset.map(tokenize_fn_bert,batched=True)
tokenized_dataset_valid_bert = validset.map(tokenize_fn_bert,batched=True)
tokenized_dataset_test_bert = testset.map(tokenize_fn_bert,batched=True)

In [ ]:
tokenized_dataset_train_bert

In [ ]:
tokenized_dataset_train_bert = tokenized_dataset_train_bert.remove_columns(["texts"])
tokenized_dataset_valid_bert =tokenized_dataset_valid_bert.remove_columns(["texts"])
tokenized_dataset_test_bert = tokenized_dataset_test_bert.remove_columns(["texts"])

In [ ]:
tokenized_dataset_train_bert

In [ ]:
tokenized_dataset_train_bert.set_format(type='torch')
tokenized_dataset_valid_bert.set_format(type='torch')
tokenized_dataset_test_bert.set_format(type='torch')

In [ ]:
tokenized_dataset_train_bert.features

In [ ]:
tokenized_dataset_test_bert

##Model for distil-bert

In [ ]:
model_distil_bert = AutoModelForSequenceClassification.from_pretrained(checkpoint_distil_bert,num_labels = 11,problem_type="multi_label_classification",id2label=id2labels,label2id=labels2id)

In [ ]:
import evaluate
from transformers import EvalPrediction
import torch
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

##Metrics labelling functions

In [ ]:
def multi_label_metrics(predictions, labels, threshold=0.5):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1': f1_micro_average,'Accuracy': accuracy}
    return metrics

def compute_metrics_1(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,tuple) else p.predictions
    result = multi_label_metrics(predictions=preds,labels=p.label_ids)
    return result

##Training Args

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    learning_rate=0.00001,
    optim='adamw_torch',
    remove_unused_columns=False,
    output_dir=str(modelfolder),
    evaluation_strategy='steps',
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_strategy='steps',
    logging_steps=10,
    report_to='wandb',
    run_name='emoticon-distilbert-1-without emoji',
)

##Bert Trainer

In [ ]:
trainer_distil_bert = Trainer(
    model=model_distil_bert,
    args=training_args,
    train_dataset = tokenized_dataset_train_bert,
    eval_dataset = tokenized_dataset_valid_bert,
    compute_metrics=compute_metrics_1,
)


In [ ]:
!wandb login 5b0f2851a77f589c556f16598a0b181257a541e9

In [ ]:
%env WANDB_PROJECT = nlp_course_spring_2024-emtoicons-analysis-hf-trainer

##Training

In [ ]:
trainer_distil_bert.train()

##Evaluation on validation data set

In [ ]:
best_model_bert_checkpoint_step = trainer_distil_bert.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_bert_checkpoint_step}.")

In [ ]:
wandb.finish()

##Save the model

In [ ]:
model_bert_checkpoint = str(modelfolder/f'checkpoint-{best_model_bert_checkpoint_step}')
model_bert_checkpoint

In [ ]:
model_distil_bert = AutoModelForSequenceClassification.from_pretrained(model_bert_checkpoint)

##Test set

In [ ]:
training_args_bert_test = TrainingArguments(
    output_dir=str(modelfolder),
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True,
    report_to='wandb',
    run_name='emoticon_hf_trainer',
)

In [ ]:
tokenized_dataset_test_bert["labels"][:5]

In [ ]:
trainer_bert_test = Trainer(
    model=model_distil_bert,
    args=training_args_bert_test,
    eval_dataset=tokenized_dataset_test_bert,
    compute_metrics=compute_metrics_1,)

In [ ]:
bert_test_results = trainer_bert_test.evaluate()

In [ ]:
bert_test_results

In [ ]:
wandb.log({"test_accuracy": bert_test_results["eval_Accuracy"], "test_loss": bert_test_results["eval_loss"], "test_f1": bert_test_results["eval_f1"]})

In [ ]:
bert_test_output = trainer_bert_test.predict(tokenized_dataset_test_bert)
bert_test_output

In [ ]:
bert_test_preds = bert_test_output.predictions

In [ ]:
bert_test_preds = torch.tensor(bert_test_preds)

##Predictions to Probablities

In [ ]:
def coversion(predictions, threshold=0.25):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    results = np.zeros(probs.shape)
    results[np.where(probs >= threshold)] = 1
    return results

In [ ]:
bert_test_preds_final = coversion(bert_test_preds)
bert_test_preds_final

In [ ]:
df_test = pd.read_csv(datafolder/"emoticons_test.csv")

In [ ]:
df_test[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
       'optimism', 'pessimism', 'sadness', 'surprise', 'trust']] = bert_test_preds_final

In [ ]:
df_test.drop(["Tweet"],axis =1, inplace = True)

In [ ]:
df_test.to_csv(basefolder/"Class_8/Home_Work_6/Submissions/bert-test.csv")

##Inference

In [ ]:
mapped_labels =[]
for prediction_row in bert_test_preds_final:
    mapped_row = [label for label, pred in zip(labels, prediction_row) if pred]
    mapped_labels.append(mapped_row)

In [ ]:
from pprint import pprint
pprint(mapped_labels[:10])